<a href="https://colab.research.google.com/github/TomasRojass/TP-Progra-concurrente/blob/main/progra_concurrente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico N° 1- Parte1

## Python

In [ ]:
%%writefile fork.py

import os
import sys
import time


def pids_proceso(letra):
    print(f"Proceso {letra}. PID {os.getpid()}, PID PADRE {os.getppid()} \n")

# tuve que crear un archivo para ver el pid para el pstree porque aca en el colab
# ni con sys.stdout.flush() me manda los prints a la terminal hasta que termina de ejecutar
os.system("rm PID_*")
with open(f"PID_{os.getpid()}", "w") as f:
  pass

pid_ab = os.fork()

if pid_ab < 0:
    sys.exit("Error al crear el nuevo proceso")

if pid_ab:  # A
    pid_ac = os.fork()
    if pid_ac:  # A
        pids_proceso("A")
        print(os.wait())  # A waitfor B
        print(os.wait())  # A waitfor C
    else:  # C
        pid_cf = os.fork()
        if pid_cf:  # C
            pids_proceso("C")
            os.wait()  # C waitfor F
            os._exit(os.EX_OK)
        else:  # F
            pids_proceso("F")
            time.sleep(30)
            os._exit(os.EX_OK)

else:  # B
    pid_bd = os.fork()
    if pid_bd:  # B
        pid_be = os.fork()
        if pid_be:  # B
            pids_proceso("B")
            print(os.wait())  # B waitfor D
            print(os.wait())  # B waitfor E
            os._exit(os.EX_OK)
        else:  # E
            pid_eg = os.fork()
            if pid_eg:  # E
                pid_eh = os.fork()
                if pid_eh:  # E
                    pids_proceso("E")
                    print(os.wait())  # E waitfor G
                    print(os.wait())  # E waitfor G
                    os._exit(os.EX_OK)
                else:
                    pids_proceso("H")
                    time.sleep(30)
                    os._exit(os.EX_OK)
            else:  # G
                pids_proceso("G")
                time.sleep(30)
                os._exit(os.EX_OK)
    else:  # D
        pids_proceso("D")
        time.sleep(30)
        os._exit(os.EX_OK)

Writing fork.py


In [ ]:
!nohup python fork.py 1>salida.txt 2>/dev/null &

In [ ]:
!pstree -pc 4282

python3(4282)─┬─python3(4286)─┬─python3(4288)
              │               └─python3(4289)─┬─python3(4291)
              │                               └─python3(4292)
              └─python3(4287)───python3(4290)


## Java:

In [ ]:
%%writefile Process.java
import java.io.IOException;

class ParentProcess
{
    static Process childs[];

    public static void createChilds() throws IOException,InterruptedException
    {
            //Proceso B
            ProcessBuilder processB = new ProcessBuilder("java", "ChildProcess.java", "B");
            processB.inheritIO();
            Process pB = processB.start();

            // Proceso C
            ProcessBuilder processC = new ProcessBuilder("java", "ChildProcess.java", "C");
            processC.inheritIO();
            Process pC = processC.start();

            pB.waitFor();
            pC.waitFor();
    }

    public static void main(String[] args) throws IOException, InterruptedException
    {
        ProcessHandle yo = ProcessHandle.current();
        long pid  = yo.pid();
        long ppid = yo.parent().get().pid();
        System.out.println("A" + " PID: " + pid + " PPID: " + ppid );
        createChilds();
    }

}

Writing Process.java


In [ ]:
%%writefile ChildProcess.java
import java.io.IOException;

class ChildProcess
{
    public static void showInfo( String id )
    {
        ProcessHandle yo = ProcessHandle.current();
        long pid  = yo.pid();
        long ppid = yo.parent().get().pid();
        System.out.println(id + " PID: " + pid + " PPID: " + ppid );
    }

    public static void main(String[] args) throws IOException,InterruptedException
    {
        showInfo(args[0]);
        if(args[0].equals("B"))
        {
            //Proceso D
            ProcessBuilder processD = new ProcessBuilder("java", "ChildProcess.java", "D");
            processD.inheritIO();
            Process pD = processD.start();

            //Proceso E
            ProcessBuilder processE = new ProcessBuilder("java", "ChildProcess.java", "E");
            processE.inheritIO();
            Process pE = processE.start();

            pD.waitFor();
            pE.waitFor();
        }

        if(args[0].equals("C"))
        {
            //Proceso F
            ProcessBuilder processF = new ProcessBuilder("java", "ChildProcess.java", "F");
            processF.inheritIO();
            Process pF = processF.start();

            pF.waitFor();
        }

        if(args[0].equals("E"))
        {
            //Proceso G
            ProcessBuilder processG = new ProcessBuilder("java", "ChildProcess.java", "G");
            processG.inheritIO();
            Process pG = processG.start();

            //Proceso H
            ProcessBuilder processH = new ProcessBuilder("java", "ChildProcess.java", "H");
            processH.inheritIO();
            Process pH = processH.start();

            pG.waitFor();
            pH.waitFor();
        }

        if(args[0].equals("G"))
          Thread.sleep(100000); // 100 segundos

        if(args[0].equals("H"))
          Thread.sleep(100000); // 100 segundos

        if(args[0].equals("D"))
          Thread.sleep(100000); // 100 segundos

        if(args[0].equals("F"))
          Thread.sleep(100000); // 100 segundos
    }
}

Writing ChildProcess.java


In [ ]:
!nohup java Process.java 1>salida.txt 2>/dev/null &

In [ ]:
!pstree -pT 1390

java(1390)─┬─java(1413)─┬─java(1460)
           │            └─java(1466)─┬─java(1536)
           │                         └─java(1539)
           └─java(1415)───java(1500)


## C++

In [ ]:
%%writefile fork.cpp
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>

char fork_wait(char process, int child_count, char childs[]);

int main()
{
  char childs_a[] = {'B', 'C'};
  char childs_b[] = {'D', 'E'};
  char childs_c[] = {'F'};
  char childs_e[] = {'G', 'H'};

  printf("Proceso A PID: %d, PPID: %d\n", getpid(), getppid());

	char process = fork_wait('A', 2, childs_a);
  if(process == 'B')
  {
    printf("Proceso B PID: %d, PPID: %d\n", getpid(), getppid());
    char process = fork_wait('B', 2, childs_b);
    if(process == 'D')
    {
      printf("Proceso D PID: %d, PPID: %d\n", getpid(), getppid());
      sleep(10);
      return EXIT_SUCCESS;
    }
    if(process == 'E')
    {
      printf("Proceso E PID: %d, PPID: %d\n", getpid(), getppid());
      char process = fork_wait('E', 2, childs_e);
      if(process == 'G' || process == 'H')
      {
        printf("Proceso %c PID: %d, PPID: %d\n", process, getpid(), getppid());
        sleep(10);
        return EXIT_SUCCESS;
      }
    }
  }
  if(process == 'C')
  {
    printf("Proceso C PID: %d, PPID: %d\n", getpid(), getppid());
    char process = fork_wait('C', 1, childs_c);
    if(process == 'F')
    {
      printf("Proceso F PID: %d, PPID: %d\n", getpid(), getppid());
      sleep(10);
      return EXIT_SUCCESS;
    }
  }
	return EXIT_SUCCESS;
}

char fork_wait(char process, int child_count, char childs[])
{
  printf("Process %c PID: %d, PPID: %d\n",process, getpid(), getppid());

  for(int child=0; child<child_count; child++)
  {
    pid_t pid = fork();
    if(!pid)
    {
      return childs[child];
    }
  }
  for(int child=0; child<child_count; child++)
  {
    wait(NULL);
  }
  return process;
}

Overwriting fork.cpp


In [ ]:
!gcc -o forkc ./fork.cpp

In [ ]:
!nohup ./forkc 1>salida-c.txt 2>/dev/null &

In [ ]:
!pstree -pc 83170

forkc(83170)─┬─forkc(83171)─┬─forkc(83175)
             │              └─forkc(83176)─┬─forkc(83177)
             │                             └─forkc(83178)
             └─forkc(83172)───forkc(83173)
